In [1]:
import hopsworks
import great_expectations as ge
from great_expectations.core import ExpectationSuite, ExpectationConfiguration

import math
import pandas as pd

from features.transactions import calculate_time_delta_t_minus_1, calculate_loc_delta_t_minus_1

In [20]:
# Connect to the Hopsworks Feature Store
project = hopsworks.login()
fs = project.get_feature_store()

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://pocs.cloud.hopsworks.ai/p/125
Connected. Call `.close()` to terminate connection gracefully.


In [21]:
# Read the historical transaction data
transactions_pdf = pd.read_csv("../../../../../RawData/historical_transactions.csv", parse_dates=['datetime'])

In [22]:
transactions_pdf = transactions_pdf[["datetime", "cc_num", "latitude", 'longitude']]

# cc_num is more of an account_id rather than a proper credit card number, rename the column to avoid confusion
transactions_pdf.rename(columns={'cc_num': 'account_id', 'datetime': 'last_transaction_datetime'}, inplace=True) 

In [23]:
profiles_last_transaction_pdf = transactions_pdf.sort_values('last_transaction_datetime').groupby('account_id').tail(1)

In [24]:
profiles_last_transaction_pdf

,last_transaction_datetime,account_id,latitude,longitude
45674,2024-03-12 15:13:58,9fb0eada8456d27aef2b641ed31b4e2d,46.09273,-88.64235
48035,2024-03-20 16:05:39,76e9b3d0385f477d02528e22ec255d03,36.91023,-121.75689
48281,2024-03-21 14:16:35,f4b5cff50c3c7af1b1c95376b5f4c834,40.57788,-73.95958
48727,2024-03-23 00:59:11,46408cecc8bc7451058ee26b2219abda,36.61033,-88.31476
48955,2024-03-23 18:03:28,97fcd8df94c0e076a89ae7099fa37e69,41.15376,-87.88754
...,...,...,...,...
53995,2024-04-09 16:56:01,2dd9b007dc4752ed27c551e281b686e9,38.17492,-122.26080
53996,2024-04-09 17:21:21,4e425702ea0aab25bdbfca71bd15a8f9,42.24113,-88.31620
53997,2024-04-09 17:36:31,ee20ed61c7108f9ca4ce4b8c4ba85bc5,45.16024,-93.08883
53998,2024-04-09 17:39:34,6a4dea5846b81579f040f3b2eed419ae,34.25628,-78.04471


In [25]:
# Create the feature group metadata
profiles_last_transaction_fg = fs.get_or_create_feature_group(
    name="profiles_last_transaction",
    version=1,
    description="Last transaction time and location for each account",
    primary_key=['account_id'],
    event_time='last_transaction_datetime',
    online_enabled=True,
    statistics_config={'histograms': True, 'correlations': True}
)

In [26]:
# Insert data into feature group
profiles_last_transaction_fg.insert(profiles_last_transaction_pdf)

Feature Group created successfully, explore it at 
https://pocs.cloud.hopsworks.ai/p/125/fs/73/fg/56


Uploading Dataframe: 0.00% |          | Rows 0/1000 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: profiles_last_transaction_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://pocs.cloud.hopsworks.ai/p/125/jobs/named/profiles_last_transaction_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x7f0ae6263040>, None)